In [0]:
storage_account_name = "adlsstorageaccount1048"
container_name = "powerconsumptiondatafiles"
file_name = "household_power_consumption.csv"
storage_account_access_key = "fJXtlCwAnbpM9XbLvcQdHWzhpeWCD+SadnM5RYB9tDFIQBOS1bo/KLkBCxyyRtMaLZvGkv3MkPZG+AStpUNMEw=="
spark.conf.set("fs.azure.account.key." + storage_account_name + ".blob.core.windows.net", storage_account_access_key)
data = spark.read.format("csv").load(f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/{file_name}", inferSchema=True, header=True)
data.show()

+----------+-------------------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|      Date|               Time|Global_active_power|Global_reactive_power|Voltage|Global_intensity|Sub_metering_1|Sub_metering_2|Sub_metering_3|
+----------+-------------------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|2007-01-01|2024-02-26 00:00:00|               2.58|                0.136| 241.97|            10.6|             0|             0|             0|
|2007-01-01|2024-02-26 00:01:00|              2.552|                  0.1| 241.75|            10.4|             0|             0|             0|
|2007-01-01|2024-02-26 00:02:00|               2.55|                  0.1| 241.64|            10.4|             0|             0|             0|
|2007-01-01|2024-02-26 00:03:00|               2.55|                  0.1| 241.71|            10.4|             0|             0| 

In [0]:
from pyspark.sql.functions import round

cleaned_data = data.dropna(subset=["Global_active_power"])  
cleaned_data.show()

+----------+-------------------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|      Date|               Time|Global_active_power|Global_reactive_power|Voltage|Global_intensity|Sub_metering_1|Sub_metering_2|Sub_metering_3|
+----------+-------------------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|2007-01-01|2024-02-26 00:00:00|               2.58|                0.136| 241.97|            10.6|             0|             0|             0|
|2007-01-01|2024-02-26 00:01:00|              2.552|                  0.1| 241.75|            10.4|             0|             0|             0|
|2007-01-01|2024-02-26 00:02:00|               2.55|                  0.1| 241.64|            10.4|             0|             0|             0|
|2007-01-01|2024-02-26 00:03:00|               2.55|                  0.1| 241.71|            10.4|             0|             0| 

In [0]:
# Drop rows with any missing date values
cleaned_data = cleaned_data.drop("Sub_metering_1")
cleaned_data = cleaned_data.drop("Sub_metering_2")
cleaned_data = cleaned_data.drop("Sub_metering_3")

cleaned_data.show()

+----------+-------------------+-------------------+---------------------+-------+----------------+
|      Date|               Time|Global_active_power|Global_reactive_power|Voltage|Global_intensity|
+----------+-------------------+-------------------+---------------------+-------+----------------+
|2007-01-01|2024-02-26 00:00:00|               2.58|                0.136| 241.97|            10.6|
|2007-01-01|2024-02-26 00:01:00|              2.552|                  0.1| 241.75|            10.4|
|2007-01-01|2024-02-26 00:02:00|               2.55|                  0.1| 241.64|            10.4|
|2007-01-01|2024-02-26 00:03:00|               2.55|                  0.1| 241.71|            10.4|
|2007-01-01|2024-02-26 00:04:00|              2.554|                  0.1| 241.98|            10.4|
|2007-01-01|2024-02-26 00:05:00|               2.55|                  0.1| 241.83|            10.4|
|2007-01-01|2024-02-26 00:06:00|              2.534|                0.096| 241.07|            10.4|


In [0]:
# Remove Duplicates
cleaned_data = cleaned_data.dropDuplicates()
cleaned_data.show()

+----------+-------------------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|      Date|               Time|Global_active_power|Global_reactive_power|Voltage|Global_intensity|Sub_metering_1|Sub_metering_2|Sub_metering_3|
+----------+-------------------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|2007-01-01|2024-02-26 02:50:00|              2.586|                0.124| 243.02|            10.6|             0|             1|             0|
|2007-01-01|2024-02-26 02:55:00|               2.69|                0.262| 243.11|              11|             0|             1|             0|
|2007-01-01|2024-02-26 07:43:00|              2.444|                 0.11| 240.25|            10.2|             0|             1|             0|
|2007-01-01|2024-02-26 11:37:00|              2.534|                    0| 238.16|            10.6|             0|             0| 

In [0]:
# Handle inconsistencies
# Convert decimals to only 3 digits

cleaned_data = cleaned_data.withColumn("Global_active_power", round((cleaned_data["Global_active_power"]),3))
cleaned_data.show()

+----------+-------------------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|      Date|               Time|Global_active_power|Global_reactive_power|Voltage|Global_intensity|Sub_metering_1|Sub_metering_2|Sub_metering_3|
+----------+-------------------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|2007-01-01|2024-02-26 02:50:00|              2.586|                0.124| 243.02|            10.6|             0|             1|             0|
|2007-01-01|2024-02-26 02:55:00|               2.69|                0.262| 243.11|              11|             0|             1|             0|
|2007-01-01|2024-02-26 07:43:00|              2.444|                 0.11| 240.25|            10.2|             0|             1|             0|
|2007-01-01|2024-02-26 11:37:00|              2.534|                    0| 238.16|            10.6|             0|             0| 